# Setup

In [ ]:
!pip install datasets -qqq
!pip install git+https://github.com/huggingface/transformers -qqq
!pip install wandb -qqq
!pip install accelerate -qqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.5/258.5 kB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 6.4 MB/s eta 0:00:00


# First Model: Populating our Model Registry

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b", device_map="auto", torch_dtype=torch.float16)

tokenizer_config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
model.save_pretrained("./models/baseline")

In [ ]:
import wandb

run = wandb.init(project="enterprise_model_management_wandb")
run.log_model(path="./models/baseline", name="baseline", aliases="baseline")
run.finish()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


wandb: Adding directory to artifact (./models/baseline)... Done. 64.8s


In [ ]:
import wandb

In [ ]:
run = wandb.init(project="enterprise_model_management_wandb")

# Clones down the artifact and returns a path.
# artifact_name = "baseline:v0"
# model_path = run.use_model(artifact_name)

# Logs as artifact + registers the model to a model registry as a model version.
model_path = "./models/baseline"

run.link_model(path=model_path, registered_model_name="gemma-2b")
run.finish()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


wandb: Adding directory to artifact (./models/baseline)... Done. 73.1s
wandb: Waiting for artifact run-rrce0npv-baseline to be committed...



wandb: Committed artifact vincenttu/enterprise_model_management_wandb/run-rrce0npv-baseline:v0


# Adding More Models: Populating our Model Registry

In [ ]:
import torch
import wandb
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it", device_map="auto", torch_dtype=torch.float16)

model.save_pretrained("./models/2B_IT")

tokenizer_config.json:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/888 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
run = wandb.init(project="enterprise_model_management_wandb")

# Logs as artifact + registers the model to a model registry as a model version.
model_path = "./models/2B_IT"

# Ref: https://docs.wandb.ai/ref/python/run#link_model.
run.link_model(path=model_path, registered_model_name="gemma-2b", aliases=["staging"])  # A common alias. Another one is "production".
run.finish()

wandb: Currently logged in as: vincenttu. Use `wandb login --relogin` to force relogin


wandb: Adding directory to artifact (./models/2B_IT)... Done. 62.3s
wandb: Waiting for artifact run-hzmp9nd4-2B_IT to be committed...



wandb: Committed artifact vincenttu/enterprise_model_management_wandb/run-hzmp9nd4-2B_IT:v0


In [ ]:
import wandb
run = wandb.init()
artifact = run.use_artifact('vincenttu/model-registry/gemma-2b:v1', type='model')
artifact_dir = artifact.download()

wandb: Downloading large artifact gemma-2b:v1, 4780.18MB. 5 files... 
wandb:   5 of 5 files downloaded.  
Done. 0:0:42.0
